<a href="https://colab.research.google.com/github/tanyag7271/fucn_call_base/blob/main/GeminiFunctionCalling_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Gemini Function Calling

This notebook follows along the required code to complete the lab on [How to Interact with APIs Using Function Calling in Gemini
](https://codelabs.developers.google.com/codelabs/gemini-function-calling) by [@koverholt](https://github.com/koverholt).

## Step 1. Overview
This section talks about the importance of Function Calling and how it enables Gemini to access data that may be real-time, protected or otherwise unavailable in the Gemini training datasets.

## Step 2. Setup and requirements

In [ ]:
%%capture
!pip install --upgrade google-cloud-aiplatform

## Step 3. Understand the problem

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel

In [ ]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file('gcp-adventure-x-a3fb7a36e1e6.json')

In [ ]:
vertexai.init(project="gcp-adventure-x", location="us-central1", credentials=credentials)

In [ ]:
model = GenerativeModel("gemini-1.0-pro-001")

In [ ]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information and cannot provide the current exchange rate. Please check a currency converter or financial news source for the most up-to-date information.


## Step 4: Try common workarounds
Skipping this section in interest of getting ideas from the workshop participants.

## Step 5. How function calling works
This section describes the execution flow of Function Calling.

## Step 6. Choose your API

In [ ]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
data = response.json()
data

{'amount': 1.0,
 'base': 'EUR',
 'date': '2024-03-15',
 'rates': {'AUD': 1.6579,
  'BGN': 1.9558,
  'BRL': 5.4461,
  'CAD': 1.4731,
  'CHF': 0.9613,
  'CNY': 7.838,
  'CZK': 25.166,
  'DKK': 7.4571,
  'GBP': 0.8541,
  'HKD': 8.5199,
  'HUF': 393.2,
  'IDR': 17011,
  'ILS': 3.9811,
  'INR': 90.26,
  'ISK': 148.9,
  'JPY': 162.03,
  'KRW': 1448.71,
  'MXN': 18.1915,
  'MYR': 5.1241,
  'NOK': 11.5205,
  'NZD': 1.786,
  'PHP': 60.494,
  'PLN': 4.2953,
  'RON': 4.9711,
  'SEK': 11.2674,
  'SGD': 1.4562,
  'THB': 39.053,
  'TRY': 35.092,
  'USD': 1.0892,
  'ZAR': 20.352}}

## Step 7. Define a function and tool

In [ ]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.0-pro-001")

In [ ]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [ ]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

## Step 8. Generate a function call

In [ ]:
prompt = """What is the latest exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [ ]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
    }
  }
}



## Step 9. Make an API request

In [ ]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'AUD', 'date': 'latest', 'to': 'SEK'}

In [ ]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.json()

{'amount': 1.0, 'base': 'AUD', 'date': '2024-03-15', 'rates': {'SEK': 6.7962}}

## Step 10. Generate a response

In [ ]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'500 Australian dollars is worth 3398.1 Swedish krona as of 2024-03-15. The exchange rate is 1 AUD = 6.7962 SEK.'

## Conclusion

Explore further:


1.   [Gemini Pro API Reference](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini#gemini-pro)
2.   [Sample Function Calling Notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb)



In [5]:
print("-"*150)

------------------------------------------------------------------------------------------------------------------------------------------------------




*  Task 1: Ask Gemini base version current currency conversion rates using function call
*  Task 2: Enquire about real-time weather conditions of different cities/countries




In [18]:
# Import VertexAI API
import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool
)

In [8]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file('thematic-runner-417406-31176b83189a.json')
vertexai.init(project="thematic-runner-417406", location="us-central1", credentials=credentials)

In [9]:
# call generative model Gemini 1.0

model = GenerativeModel("gemini-1.0-pro")

In [11]:
%%time
response = model.generate_content("What is the meanining of large language models?")
print(response.text)

**Large Language Models (LLMs)**

LLMs are a type of artificial intelligence (AI) that specialize in processing and generating large amounts of text data. They are trained on massive datasets of text, typically hundreds of billions or even trillions of words, enabling them to understand and produce human-like text.

**Characteristics of LLMs:**

* **Large-scale:** LLMs are trained on the most extensive text datasets available, giving them a vast vocabulary and comprehensive understanding of language.
* **Generative:** LLMs can generate new text, including stories, poems, articles, and dialogue.
* **Contextual:** LLMs can consider the context of the input text to produce coherent and meaningful responses.
* **Multimodal:** Some LLMs can also process and generate other types of data, such as images, sound, and video.

**Applications of LLMs:**

LLMs have a wide range of potential applications, including:

* **Natural language processing (NLP):** LLMs improve the accuracy and efficiency o

In [15]:
# LLM is able to answer ques like above, but when asked about real-time or current information, it gives a response with outdated information
response = model.generate_content("what is the weather in paris like today?")
print(response.text)

I do not have real-time capabilities and my knowledge cutoff is April 2023. Therefore, I cannot provide you with the current weather conditions in Paris.

To get the most up-to-date and accurate weather forecast, I recommend checking a reputable weather app or website, such as the following:

* **The Weather Channel:** https://weather.com/
* **AccuWeather:** https://www.accuweather.com/
* **Meteorologie Nationale (French National Weather Service):** https://meteofrance.com/

These sources will provide you with the latest weather conditions, including temperature, humidity, wind speed, and precipitation probability.


In [ ]:
# In cases like these, you could call an external API then feed the results back to the generative model for it to use in its response
# Step 1: Specify the API
# Step 2: Function Declaration & Tool
# Step 3: Generate a function call on behalf of LLM

In [17]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.json()

{'amount': 1.0,
 'base': 'EUR',
 'date': '2024-03-15',
 'rates': {'AUD': 1.6579,
  'BGN': 1.9558,
  'BRL': 5.4461,
  'CAD': 1.4731,
  'CHF': 0.9613,
  'CNY': 7.838,
  'CZK': 25.166,
  'DKK': 7.4571,
  'GBP': 0.8541,
  'HKD': 8.5199,
  'HUF': 393.2,
  'IDR': 17011,
  'ILS': 3.9811,
  'INR': 90.26,
  'ISK': 148.9,
  'JPY': 162.03,
  'KRW': 1448.71,
  'MXN': 18.1915,
  'MYR': 5.1241,
  'NOK': 11.5205,
  'NZD': 1.786,
  'PHP': 60.494,
  'PLN': 4.2953,
  'RON': 4.9711,
  'SEK': 11.2674,
  'SGD': 1.4562,
  'THB': 39.053,
  'TRY': 35.092,
  'USD': 1.0892,
  'ZAR': 20.352}}

In [24]:
# function declaration -----> Tool
get_exch_rate_function = FunctionDeclaration(
    name= 'get_exchange_rate',
    description="Get currency exchange rate between countries",
    parameters= {
        "type": "object",
        "properties": {
            "date": {
                "type": "string",
                "description": "A date must always be in 'YYYY-MM-DD' format or current date if value is not given"
            },
            "from": {
                "type": "string",
                "description": "Which currency to convert from"
            },
            "to": {
                "type": "string",
                "description": "Which currency to convert into"
            }
        },
        "required": ["from",
                     "to"]
    },
)

# register one or more function declarations in a tool
exch_rate_tool = Tool(function_declarations=[get_exch_rate_function],
                      )

In [29]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exch_rate_tool],
)

InternalServerError: 500 Internal error encountered.

In [30]:
prompt = "What is the current conversion rate from Australian dollars to Indian Rupees?"

response = model.generate_content(prompt,
                                  tools=[exch_rate_tool],
                                  )


InternalServerError: 500 Internal error encountered.

In [ ]:
print(response.candidates[0].content)